In [1]:
import pandas as pd

df = pd.read_csv('../amlo_complete.csv')
#df = df.head(50)
#df.reset_index(inplace=True)

df['new_classification'] = ''
df

,Texto,Clasificacion
0,Buenos días y feliz año. Deseamos que nos vay...,saludos
1,Vamos a iniciar la semana como lo hacemos siem...,NaN
2,También vamos a informar este primer día hábil...,apoyo
3,Y ahora se va a informar porque se está optand...,apoyo
4,En esta etapa se combinan los bancos particula...,apoyo


In [2]:
import os
with open(r'C:\Users\claud\Documents\Descargas\open_ai_key.txt') as f:
    os.environ["OPENAI_API_KEY"] = f.readline()

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name= "gpt-3.5-turbo")


from langchain.schema.messages import HumanMessage, SystemMessage
prompt='''

You are a virtual assistant, who only knows how to classify texts in Spanish and responds only in Spanish, he cannot respond in English.

The classifications that you should take into account are:
Security: talk about security in Mexico
History: facts from the history of Mexico or the world, no comments.
Economy: Refers to the economic situation in Mexico.
Foreign: Talk about how other countries get involved with Mexico or how Mexico impacts abroad, also about migration.
Opinion: People's personal opinion on any topic, mostly opinions about history in the actual society.
Health: Health situation in Mexico.
Support: social and financial support to communities, older adults, indigenous people, etc.
Corruption: About the wrongdoings of other groups or organizations.
Opposition: Opposed to certain ideas or thoughts, even opposition or badmouthing about a person, organization or Politics People.
Construction: Infrastructure built or purchased for the betterment of Mexico.


The answer should only be one word in Spanish for each sentence, not English.


Given you are the virtual assistant classify the following 10 sentences enclosed by quotation marks and separated with commas.
'''

,index,Texto,Clasificacion
0,0,Buenos días y feliz año. Deseamos que nos vay...,saludos
1,1,Vamos a iniciar la semana como lo hacemos siem...,NaN
2,2,También vamos a informar este primer día hábil...,apoyo
3,3,Y ahora se va a informar porque se está optand...,apoyo
4,4,En esta etapa se combinan los bancos particula...,apoyo
5,5,"También Ariadna Montiel, que es la secretaria ...",apoyo
6,6,Pero tenerlas terminadas no significa tenerlas...,apoyo
7,7,"También lo ocurrido ayer en Chihuahua, en Ciud...",seguridad
8,8,"De tomas maneras mañana, aun cuando no esté a...",NaN
9,9,"Bueno, ahora que estuvo con nosotros el gober...",seguridad


In [3]:
result = []
indice = 0
for i in range(0, len(df), 10):
    result.append('","'.join(df['Texto'].iloc[i:i+10]))
    result[indice] = '"' + result[indice]
    result[indice] = result[indice] + '"'
    indice+=1

In [ ]:
lista_de_resultados = []

In [4]:
def clasificate_csv(text):

        response = llm.invoke(
                [SystemMessage(content=prompt), HumanMessage(content=text)]
            )
        return response.content


    
for i in range(2):
    resultado_de_la_clasificacion = clasificate_csv(result[i])
    print(resultado_de_la_clasificacion)
    print(len(resultado_de_la_clasificacion))
    print(type(resultado_de_la_clasificacion))
    df.iloc[i*10:(i+1)*10, 2] = resultado_de_la_clasificacion.split(", ")
    print("Fin de la asignacion")
    
    
#df["new_clasification"] = df["Texto"].apply(clasificate_csv)


#df.to_csv('amlo_clasify_chatpgt2.csv', index=False)

In [ ]:
df.head(50)

## 1. Ahorita le mande 50 con el nuevo prompt y si los clasifico de a 10
- Opinion, Support, Support, Support, Support, Support, Support, Security, Support, Opposition
- Security, Support, Opinion, History, History, History, History, Opposition, Opposition, Opposition
- History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition.
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security.

### 2. Prompt llamado Bob
- Opinion, Health, Support, Support, Support, Construction, Construction, Security, Support, Security
- Security, History, Support, Opinion, History, History, History, Foreign, Opposition, Opposition
- History, History, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Corruption, Opposition, Opposition, Corruption, Opposition, Opinion

### 3. Prompt Bob mejorado (de mejor  no tiene nada)
13

- Opinion, Support, Support, Support, Support, Support, Support, Security, Construction, Support
- Seguridad, Apoyo, Apoyo
- Apoyo, Apoyo, Apoyo, Apoyo, Apoyo
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Historia, Historia, Historia, Historia, Historia
- Opposición, Opposición
- History, Opinion, Opinion, Opinion, Economy, Economy, Construction, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security


### 4. Prompt sin ser Bob pero mejor 

- Opinion, Support, Support, Support, Support, Construction, Construction, Security, Support, Security
- Seguridad, Apoyo, Seguridad, Historia, Historia, Historia, Historia, Historia, Historia, Historia
- History, Opinion, Opinion, Opinion, Economy, Economy, Support, Construction, Construction, Construction
- Economy, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition, Opposition
- Opinion, Opinion, Opposition, Opposition, Opposition, Opinion, Opinion, Opposition, Economy, Security

### 5. Prompt anterior pero reiterando que no puede responder en Ingles
- Opinion, Support, Health, Support, Support, Support, Support, Security, Construction, Opposition.
- Seguridad, Apoyo, Seguridad, Historia, Historia, Historia, Historia, Historia, Historia, Historia
- History, Opposition, Opinion, Opinion, Economy, Economy, Construction, Construction, Construction, Construction

In [12]:
df

,index,Texto,Clasificacion,new_clasification
0,0,Buenos días y feliz año. Deseamos que nos vay...,saludos,Apoyo
1,1,Vamos a iniciar la semana como lo hacemos siem...,NaN,Apoyo
2,2,También vamos a informar este primer día hábil...,apoyo,Support
3,3,Y ahora se va a informar porque se está optand...,apoyo,Apoyo
4,4,En esta etapa se combinan los bancos particula...,apoyo,Economía
5,5,"También Ariadna Montiel, que es la secretaria ...",apoyo,Apoyo
6,6,Pero tenerlas terminadas no significa tenerlas...,apoyo,Economy
7,7,"También lo ocurrido ayer en Chihuahua, en Ciud...",seguridad,Seguridad
8,8,"De tomas maneras mañana, aun cuando no esté a...",NaN,Apoyo
9,9,"Bueno, ahora que estuvo con nosotros el gober...",seguridad,Seguridad
